In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [3]:
from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [4]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

2021-02-13 08:56:18,650 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-13 08:56:18,651 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2021-02-13 08:56:18,651 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2021-02-13 08:56:18,653 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [6]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [10]:
help(corpora.MmCorpus)

Help on class MmCorpus in module gensim.corpora.mmcorpus:

class MmCorpus(gensim.corpora._mmreader.MmReader, gensim.corpora.indexedcorpus.IndexedCorpus)
 |  MmCorpus(fname)
 |  
 |  Corpus serialized using the `sparse coordinate Matrix Market format
 |  <https://math.nist.gov/MatrixMarket/formats.html>`_.
 |  
 |  Wrap a term-document matrix on disk (in matrix-market format), and present it
 |  as an object which supports iteration over the matrix rows (~documents).
 |  
 |  Notable instance attributes:
 |  
 |  Attributes
 |  ------------------
 |  num_docs : int
 |      Number of documents in the market matrix file.
 |  num_terms : int
 |      Number of features (terms, topics).
 |  num_nnz : int
 |      Number of non-zero elements in the sparse MM matrix.
 |  
 |  Notes
 |  -----
 |  The file is read into memory one document at a time, not the whole matrix at once,
 |  unlike e.g. `scipy.io.mmread` and other implementations. This allows you to **process corpora which are larger
 |  